In [1]:
import os

import tensorflow as tf
from tensorflow import keras

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data(path=os.getcwd() +'/data/mnist.npz')

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 8s 1us/step


In [3]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.nn.softmax)
  ])
  
  model.compile(optimizer=tf.train.AdamOptimizer(), 
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])
  
  return model


# Create a basic model instance
model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10, 
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 864/1000 [========================>.....] - ETA: 0s - loss: 1.3044 - acc: 0.6065
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 1ms/step - loss: 1.2153 - acc: 0.6340 - val_loss: 0.7993 - val_acc: 0.7440
Epoch 2/10
 768/1000 [======================>.......] - ETA: 0s - loss: 0.4544 - acc: 0.8724
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 349us/step - loss: 0.4405 - acc: 0.8780 - val_loss: 0.5586 - val_acc: 0.8360
Epoch 3/10
 832/1000 [=======================>......] - ETA: 0s - loss: 0.2860 - acc: 0.9231
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 314us/step - loss: 0.2756 - acc: 0.9280 - val_loss: 0.5045 - val_acc: 0.8390
Epoch 4/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.1958 - acc: 0.9500
Epoch 00004: saving model to training_1/cp.ckpt
1000/1000 [====

In [5]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [6]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 203us/step
Untrained model, accuracy:  8.00%


In [7]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 81us/step
Restored model, accuracy: 86.50%


In [8]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [9]:
! ls {checkpoint_dir}

checkpoint			  cp-0030.ckpt.data-00000-of-00001
cp-0005.ckpt.data-00000-of-00001  cp-0030.ckpt.index
cp-0005.ckpt.index		  cp-0035.ckpt.data-00000-of-00001
cp-0010.ckpt.data-00000-of-00001  cp-0035.ckpt.index
cp-0010.ckpt.index		  cp-0040.ckpt.data-00000-of-00001
cp-0015.ckpt.data-00000-of-00001  cp-0040.ckpt.index
cp-0015.ckpt.index		  cp-0045.ckpt.data-00000-of-00001
cp-0020.ckpt.data-00000-of-00001  cp-0045.ckpt.index
cp-0020.ckpt.index		  cp-0050.ckpt.data-00000-of-00001
cp-0025.ckpt.data-00000-of-00001  cp-0050.ckpt.index
cp-0025.ckpt.index


In [10]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [11]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 170us/step
Restored model, accuracy: 87.80%


In [12]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 192us/step
Restored model, accuracy: 87.80%


In [14]:
model = create_model()

# You need to use a keras.optimizer to restore the optimizer state from an HDF5 file.
model.compile(optimizer=keras.optimizers.Adam(), 
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('./checkpoints/my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 1s 738us/step - loss: 1.2051 - acc: 0.6460
Epoch 2/5
1000/1000 [==============================] - 0s 287us/step - loss: 0.4354 - acc: 0.8840
Epoch 3/5
1000/1000 [==============================] - 0s 294us/step - loss: 0.2907 - acc: 0.9270
Epoch 4/5
1000/1000 [==============================] - 0s 269us/step - loss: 0.2133 - acc: 0.9540
Epoch 5/5
1000/1000 [==============================] - 0s 286us/step - loss: 0.1637 - acc: 0.9660


In [16]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('./checkpoints/my_model.h5')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [17]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 319us/step
Restored model, accuracy: 86.90%


In [18]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 1s 761us/step - loss: 1.1484 - acc: 0.6930
Epoch 2/5
1000/1000 [==============================] - 0s 193us/step - loss: 0.4194 - acc: 0.8900
Epoch 3/5
1000/1000 [==============================] - 0s 206us/step - loss: 0.2865 - acc: 0.9190
Epoch 4/5
1000/1000 [==============================] - 0s 193us/step - loss: 0.2034 - acc: 0.9540
Epoch 5/5
1000/1000 [==============================] - 0s 191us/step - loss: 0.1557 - acc: 0.9640


In [19]:
saved_model_path = tf.contrib.saved_model.save_keras_model(model, "./saved_models")

INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: ['eval']
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tens

In [20]:
!ls saved_models/

1542699225


In [21]:
new_model = tf.contrib.saved_model.load_keras_model(saved_model_path)
new_model

In [22]:
# The optimizer was not restored, re-attach a new one.
new_model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 340us/step
Restored model, accuracy: 86.20%
